In [27]:
from eval_1022 import *
import json
import numpy as np

# train_val_dataset = json.load(open("/home/pcarragh/dev/webqa/MultiModalQA/data/WebQA_train_val_obj_v2.json.pert.lama.v3", "r"))

train_dataset = json.load(open("/home/pcarragh/dev/webqa/MultiModalQA/data/WebQA_train_val_obj_v2.json.pert.lama.v3", "r"))
val_dataset = json.load(open("/home/pcarragh/dev/webqa/MultiModalQA/data/WebQA_train_val_obj_v2.json.pert.lama.v3.val.gpt.json", "r"))

final_dataset = {}

val_generations = 0
train_generations = 0

val_match_generations = 0
train_match_generations = 0

# merge train and val generated images
for k, v in val_dataset.items():
    if val_dataset[k]['split'] == 'val':
        if 'A_perturbed' not in v:
            continue
        val_generations += len(v['A_perturbed'])
        for gen_label, gpt_label in zip(v['A_perturbed'].values(), v['A_perturbed_gpt'].values()):
            if gen_label[:3] == gpt_label[:3]:
                val_match_generations += 1
                final_dataset[k] = v
for k, v in train_dataset.items():
    if train_dataset[k]['split'] == 'train':
        if 'A_perturbed' not in v:
            continue
        train_generations += len(v['A_perturbed'])
        for gen_label, gpt_label in zip(v['A_perturbed'].values(), v['A_perturbed_gpt'].values()):
            if gen_label[:3] == gpt_label[:3]:
                train_match_generations += 1
                final_dataset[k] = v

print("Val generations: ", val_generations)
print("Val match generations: ", val_match_generations)
print("Train generations: ", train_generations)
print("Train match generations: ", train_match_generations)

# json.dump(final_dataset, open("/home/pcarragh/dev/webqa/MultiModalQA/data/WebQA_train_val_lama.json", "w"))


eval_data = {k:v for k, v in final_dataset.items() if v['Qcate'].lower() in ['color']}#, 'shape', 'yesno', 'number']}
len(eval_data)


Val generations:  901
Val match generations:  86
Train generations:  7751
Train match generations:  364


268

In [38]:
from llava.mm_utils import get_model_name_from_path
from llava.eval.run_llava import eval_model
# from llava.model.builder import load_pretrained_model

model_path = "liuhaotian/llava-v1.5-7b"

def llava_eval_on_webqa_sample(question, image_files, webqa_image_loader = True):
    args = type('Args', (), {
        "model_path": model_path,
        "model_base": None,
        "model_name": get_model_name_from_path(model_path),
        "query": question,
        "conv_mode": None,
        "image_file": image_files,
        "sep": ",",
        "temperature": 0,
        "top_p": None,
        "num_beams": 1,
        "max_new_tokens": 512,
        "webqa": webqa_image_loader,
    })()

    return eval_model(args)

def webqa_accuracy(answer, label, Qcate):
    if Qcate == 'color':
        F1_avg, F1_max, EM, RE_avg, PR_avg = compute_vqa_metrics([answer], label[0], "", color_set)
    elif Qcate == 'shape': 
        F1_avg, F1_max, EM, RE_avg, PR_avg = compute_vqa_metrics([answer], label[0], "", shape_set)
    elif Qcate == 'yesno': 
        F1_avg, F1_max, EM, RE_avg, PR_avg = compute_vqa_metrics([answer], label[0], "", yesno_set)
    elif Qcate == 'number': 
        F1_avg, F1_max, EM, RE_avg, PR_avg = compute_vqa_metrics([answer], label[0], "", {"NUMBER"})
    else:
        return None
    return (F1_avg, F1_max, EM, RE_avg, PR_avg)

def accuracy_agg_results(qa_results):
    single_image_keys = [k for k in qa_results.keys() if len(eval_data[k]['img_posFacts']) == 1]
    two_image_keys = [k for k in qa_results.keys() if len(eval_data[k]['img_posFacts']) == 2]

    single_acc = np.mean([PR_avg for key, (F1_avg, F1_max, EM, RE_avg, PR_avg) in qa_results.items() if key in single_image_keys])
    two_image_acc = np.mean([PR_avg for key, (F1_avg, F1_max, EM, RE_avg, PR_avg) in qa_results.items() if key in two_image_keys])
    avr_acc = np.mean([PR_avg for key, (F1_avg, F1_max, EM, RE_avg, PR_avg) in qa_results.items()])
    return (single_acc, two_image_acc, avr_acc)

### LLAVA baseline on original images that have perturbations

In [28]:
llava_results_baseline = {}
for k in eval_data.keys():
    question = eval_data[k]['Q']
    image_files = ','.join([str(img_data['image_id']) for img_data in eval_data[k]['img_posFacts']])
    answer = llava_eval_on_webqa_sample(question, image_files, True)
    label = eval_data[k]['A']
    eval_data[k]['A_llava'] = answer
    Qcate = eval_data[k]['Qcate'].lower()
    llava_results_baseline[k] = webqa_accuracy(answer, label, Qcate)

print(accuracy_agg_results(llava_results_baseline))

/home/pcarragh/miniconda3/envs/llava/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You are using a model of type llava to instantiate a model of type llava_llama. This is not supported for all configurations of models and can yield errors.
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.77s/it]
/home/pcarragh/miniconda3/envs/llava/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/pcarragh/miniconda3/envs/llava/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:397: UserWarnin

(0.2846698113207547, 0.34851190476190474, 0.29800995024875626)


### Correct despite blank image

In [29]:
llava_results_blank = {}
for k in eval_data.keys():
    question = eval_data[k]['Q']
    image_files ='playground/counterfactual_exp/BLANK.jpg'
    answer = llava_eval_on_webqa_sample(question, image_files, False)

    label = eval_data[k]['A']
    eval_data[k]['A_llava_blank'] = answer
    Qcate = eval_data[k]['Qcate'].lower()
    llava_results_blank[k] = webqa_accuracy(answer, label, Qcate)
print(accuracy_agg_results(llava_results_blank))

/home/pcarragh/miniconda3/envs/llava/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You are using a model of type llava to instantiate a model of type llava_llama. This is not supported for all configurations of models and can yield errors.
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.91s/it]
/home/pcarragh/miniconda3/envs/llava/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/pcarragh/miniconda3/envs/llava/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:397: UserWarnin

(0.2540094339622641, 0.3306547619047619, 0.27002487562189054)


### Correct despite perturbation

In [ ]:
llava_results_perturbed_original_label = {}
llava_results_perturbed_generated_label = {}

for k in eval_data.keys():
    llava_results_perturbed_original_label[k] = {}
    llava_results_perturbed_generated_label[k] = {}
    eval_data[k]['A_perturbed_llava'] = {}
    question = eval_data[k]['Q']
    for idx, label in eval_data[k]['A_perturbed'].items():
        image_id = eval_data[k]['img_posFacts'][0]['image_id']
        # TODO: add image 2 (image loader can't handle this now)
        # image_files = ','.join([str(img_data['image_id']) for img_data in eval_data[k]['img_posFacts']])
        image_files = f"/home/pcarragh/dev/webqa/segment/Inpaint-Anything/results/webqa/{eval_data[k]['split']}/{str(image_id)}_{k}_{idx}.jpeg"
        answer = llava_eval_on_webqa_sample(question, image_files, False)

        original_label = eval_data[k]['A']
        eval_data[k]['A_perturbed_llava'][idx] = answer
        Qcate = eval_data[k]['Qcate'].lower()
        llava_results_perturbed_original_label[k][idx] = webqa_accuracy(answer, original_label, Qcate)
        llava_results_perturbed_generated_label[k][idx] = webqa_accuracy(answer, [label], Qcate)

/home/pcarragh/miniconda3/envs/llava/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You are using a model of type llava to instantiate a model of type llava_llama. This is not supported for all configurations of models and can yield errors.
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.92s/it]
/home/pcarragh/miniconda3/envs/llava/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/pcarragh/miniconda3/envs/llava/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:397: UserWarnin

In [41]:
def accuracy_agg_generated_results(qa_results):
    single_image_keys = [k for k in qa_results.keys() if len(eval_data[k]['img_posFacts']) == 1]
    two_image_keys = [k for k in qa_results.keys() if len(eval_data[k]['img_posFacts']) == 2]

    single_acc = [PR_avg for key, dict in qa_results.items() if key in single_image_keys for idx, (_,_,_,_,PR_avg) in dict.items()]
    two_image_acc = [PR_avg for key, dict in qa_results.items() if key in two_image_keys for idx, (_,_,_,_,PR_avg) in dict.items()]    
    avr_acc = [PR_avg for key, dict in qa_results.items() for idx, (_,_,_,_,PR_avg) in dict.items()]
    
    return (single_acc, two_image_acc, avr_acc)

print(accuracy_agg_generated_results(llava_results_perturbed_original_label))
print(accuracy_agg_generated_results(llava_results_perturbed_generated_label))

AttributeError: 'tuple' object has no attribute 'items'

In [39]:
import pandas as pd

baseline_accs = accuracy_agg_results(llava_results_baseline)
blank_accs = accuracy_agg_results(llava_results_blank)
perturbed_original_label_accs = accuracy_agg_generated_results(llava_results_perturbed_original_label)
perturbed_generated_label_acc = accuracy_agg_generated_results(llava_results_perturbed_generated_label)

columns = ['experiment name', 'single_image', 'two_image', 'average']
accuracy_agg_df = pd.DataFrame(columns=columns)
accuracy_agg_df['experiment name'] = ['baseline', 'blank', 'perturbed_original_label', 'perturbed_generated_label']
accuracy_agg_df['single_image'] = [baseline_accs[0], blank_accs[0], np.mean(perturbed_original_label_accs[0]), np.mean(perturbed_generated_label_acc[0])]
accuracy_agg_df['two_image'] = [baseline_accs[1], blank_accs[1], np.mean(perturbed_original_label_accs[1]), np.mean(perturbed_generated_label_acc[1])]
accuracy_agg_df['average'] = [baseline_accs[2], blank_accs[2], np.mean(perturbed_original_label_accs[2]), np.mean(perturbed_generated_label_acc[2])]
accuracy_agg_df


(0.31383647798742137, 0.3880952380952381, 0.3293532338308457)
(0.2940251572327044, 0.3851190476190477, 0.3130597014925373)


### Correct despite blank image and perturbation

In [33]:
# for questions that llava gets right regardless of having a blank image, check accuracy on counterfactual examples
perturbation_ignored = set([k for k, (_,_,_,_,acc) in llava_results_perturbed_original_label.items() if acc > 0])
blank_ignored = set([k for k, (_,_,_,_,acc) in llava_results_blank.items() if acc > 0])
parametric_ans_keys = perturbation_ignored.intersection(blank_ignored)
len(parametric_ans_keys)

40

In [37]:
for key in parametric_ans_keys:
    print(f"Q: {eval_data[key]['Q']}")
    print(f"A: {eval_data[key]['A']}")
    print(f"A_llava: {eval_data[key]['A_llava']}")
    print(f"A_llava_blank: {eval_data[key]['A_llava_blank']}")
    print(f"A_perturbed_llava: {eval_data[key]['A_perturbed_llava']}")
    print(f"A_perturbed: {eval_data[key]['A_perturbed']}, A_perturbed_gpt: {eval_data[key]['A_perturbed_gpt']}")
    # , A_llava_blank: {eval_data[key]['A_llava_blank']}, A_perturbed_llava: {eval_data[key]['A_perturbed_llava']}")

Q: "What are the colors of the Neumayer Station Antarctica building?"
A: ['"The Neumayer Station Antarctica building is in red, white, and grey."']
A_llava: The Neumayer Station Antarctica building is white and blue.
A_llava_blank: The Neumayer Station Antarctica building is predominantly white, with some black accents.
A_perturbed_llava: {'0': 'The Neumayer Station Antarctica building is painted yellow and blue.', '1': 'The Neumayer Station Antarctica building is white and blue.', '2': 'The Neumayer Station Antarctica building is painted in gold and black colors.', '3': 'The Neumayer Station Antarctica building is black and white.'}
A_perturbed: {'0': 'yellow', '1': 'gray', '2': 'gold', '3': 'purple'}, A_perturbed_gpt: {'1': 'yellow'}
Q: "What color can the different species of Amanita mushrooms be?"
A: ['"Different species of Amanita mushrooms can be white and red."']
A_llava: Amanita mushrooms can be found in various colors, including white, yellow, orange, red, and even blue. The c